In [1]:
import requests
from datetime import date
import datetime
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd

In [2]:
url = 'https://www.worldometers.info/coronavirus/'
SLEEP_TIME = 10
get_date = date.today() - datetime.timedelta(days=1)

### LẤY DỮ LIỆU

In [3]:
browser = webdriver.Chrome()
browser.get(url)
time.sleep(SLEEP_TIME)
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    # Scroll trang web xuống cuối trang.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    # Chờ để load trang web.
    time.sleep(SLEEP_TIME)
        
        # Thực hiện gán chiều cao mới sau khi kéo xuống cuối page và giảm số lần lặp nếu có chỉnh thông số time_loops
    new_height = browser.execute_script("return document.body.scrollHeight")
    if (new_height == last_height):
        break

    last_height = new_height
soup = BeautifulSoup(browser.page_source,'html.parser')
browser.close()

### XỬ LÝ DỮ LIỆU TRƯỚC KHI LƯU

In [4]:
def colname_process(s):
    n = len(s)
    for i in range(n-1):
        if('a'<=s[i]<='z' and 'A'<=s[i+1]<='Z'):
           s = s[:i+1]+' '+s[i+1:]
    return s
table = soup.find('div',{'id':'nav-yesterday'})
rows = table.find_all('tr')
colsname = [row.get_text() for row in rows[0].find_all('th')]
colsname = [colname_process(colname).strip() for colname in colsname]
rows.remove(rows[0])
for i in range(len(colsname)):
    colsname[i] = colsname[i].replace('\n','')
colsname[10] = colsname[10].replace('\xa0',' ')

In [5]:
result = []
for row in rows:
    if 'total_row_world' not in row.get('class') and 'total_row' not in row.get('class'):
        cols = row.find_all('td')
        data = [col.get_text() for col in cols]
        result.append(data)

In [6]:
def preprocess_1_row(data):
    int_comma = [0,2,4,6,8,9,10,11,12,13,14]
    new_ = [3,5,7]
    for i in int_comma:
        if type(data[i]) != int and data[i].strip() != '' and data[i] != 'N/A':
            data[i] = int(data[i].replace(',',''))
            continue
        if type(data[i]) == str and data[i].strip() == '':
            data[i] = 0
    for i in new_:
        if type(data[i]) != int and data[i].strip() != '' and data[i] != 'N/A':
            data[i] = int(data[i][1:].replace(',',''))
            continue
        if type(data[i]) == str and data[i].strip() == '':
            data[i] = 0
    return data

def preprocess_all(list_data):
    for i in range(len(list_data)):
        list_data[i] = preprocess_1_row(list_data[i])
    return list_data

In [7]:
data = preprocess_all(result)

### Write to csv

In [8]:
df = pd.DataFrame(data,columns=colsname)
df.to_csv(f'Data/{get_date}.csv',encoding='utf-8',sep=';',index=False)